# Исследование базы данных сервиса Stackoverflow

В этом исследовании работа идёт с базой данных `StackOverflow` — сервиса вопросов и ответов о программировании. `StackOverflow` похож на социальную сеть — пользователи сервиса задают вопросы, отвечают на посты, оставляют комментарии и ставят оценки другим ответам. Я исследовал предоставленные данные, сформировал требуемые выгрузки данных с помощью SQL, посчитал различные продуктовые метрики и визуализировал их.

Работа проходила на тренажере в `Яндекс практикум`и проверялась автоматически.

1.Посчитайте, сколько компаний закрылось.

In [ ]:
SELECT COUNT(id)
FROM company
WHERE status ='closed'

2.Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы `company`. Отсортируйте таблицу по убыванию значений в поле `funding_total` .

In [ ]:
SELECT funding_total
FROM company
WHERE country_code = 'USA' AND category_code = 'news'
ORDER BY funding_total DESC;

3.Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

In [ ]:
SELECT SUM(price_amount)
FROM acquisition
WHERE term_code = 'cash' AND
      CAST(acquired_at AS DATE) BETWEEN '2011-01-01' AND '2013-12-31';
      

4.Отобразите имя, фамилию и названия аккаунтов людей в твиттере, у которых названия аккаунтов начинаются на `'Silver'`.

In [ ]:
SELECT first_name,
       last_name,
       twitter_username
FROM people
WHERE twitter_username LIKE 'Silver%'

5.Выведите на экран всю информацию о людях, у которых названия аккаунтов в твиттере содержат подстроку `'money'`, а фамилия начинается на `'K'`.

In [ ]:
SELECT * 
       
FROM people
WHERE twitter_username LIKE '%money%' AND
      last_name LIKE 'K%'

6.Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.

In [ ]:
SELECT country_code,
       SUM(funding_total)
FROM company
GROUP BY country_code
ORDER BY  SUM(funding_total) DESC;

7.Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.
Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

In [ ]:
SELECT funded_at,
      MIN(raised_amount) AS min_investment,
      MAX(raised_amount) AS max_investment
FROM funding_round
GROUP BY funded_at
HAVING MIN(raised_amount) <> 0
   AND MIN(raised_amount) <> MAX(raised_amount)

8.Создайте поле с категориями:
* Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию `high_activity`.
* Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию `middle_activity`.
* Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию `low_activity`.

Отобразите все поля таблицы `fund` и новое поле с категориями.

In [ ]:
SELECT *,
       CASE
           WHEN invested_companies > 100 THEN 'high_activity'
           WHEN invested_companies BETWEEN 20 AND 100 THEN 'middle_activity'
           WHEN invested_companies < 20 THEN 'low_activity'
       END    
FROM fund;

9.Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

In [ ]:
SELECT 
       CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity,
       ROUND(AVG(investment_rounds))
FROM fund
GROUP BY activity
ORDER BY AVG(investment_rounds);


10.Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы. 

* Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. 
* Выгрузите десять самых активных стран-инвесторов: отсортируйте таблицу по среднему количеству компаний от большего к меньшему. Затем добавьте сортировку по коду страны в лексикографическом порядке.

In [ ]:
SELECT f.country_code,
       MIN(f.invested_companies) AS min_invested_companies,
       MAX(f.invested_companies) AS max_invested_companies,
       AVG(f.invested_companies) AS avg_invested_companies
FROM fund f
WHERE EXTRACT('year' FROM f.founded_at) BETWEEN 2010 AND 2012
GROUP BY f.country_code
HAVING MIN(f.invested_companies) > 0
ORDER BY avg_invested_companies DESC, f.country_code ASC
LIMIT 10;

11.Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

In [ ]:
SELECT p.first_name,
       p.last_name,
       e.instituition
FROM people AS p
LEFT JOIN education AS e ON p.id = e.person_id

12.Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.

In [ ]:
WITH comp as
(SELECT c.id,
       COUNT(DISTINCT e.instituition) AS count_instit
FROM company AS c
LEFT JOIN people AS p ON c.id = p.company_id
LEFT JOIN education AS e ON p.id = e.person_id
GROUP BY c.id)

SELECT c.name,
       count_instit
FROM comp
JOIN company as c ON comp.id = c.id
ORDER BY count_instit DESC
LIMIT 5;

13.Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

In [ ]:
SELECT name
FROM company
WHERE status = 'closed'
  AND id IN (SELECT company_id 
             FROM funding_round
             WHERE is_first_round = 1
                AND is_last_round = 1);

14.Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

In [ ]:
WITH comp AS(SELECT id
             FROM company
             WHERE status = 'closed'
               AND id IN (SELECT company_id 
                          FROM funding_round
                          WHERE is_first_round = 1
                            AND is_last_round = 1))

SELECT DISTINCT p.id
FROM comp
JOIN people AS p ON comp.id = p.company_id

15.Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

In [ ]:
WITH comp AS(SELECT id
             FROM company
             WHERE status = 'closed'
               AND id IN (SELECT company_id 
                          FROM funding_round
                          WHERE is_first_round = 1
                            AND is_last_round = 1))

SELECT DISTINCT p.id,
       e.instituition
FROM comp
JOIN people AS p ON comp.id = p.company_id
JOIN education AS e ON p.id = e.person_id;
 

16.Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.

In [ ]:
WITH comp AS(SELECT id
             FROM company
             WHERE status = 'closed'
               AND id IN (SELECT company_id 
                          FROM funding_round
                          WHERE is_first_round = 1
                            AND is_last_round = 1))

SELECT DISTINCT p.id,
       COUNT(e.instituition)
       
FROM comp
JOIN people AS p ON comp.id = p.company_id
JOIN education AS e ON p.id = e.person_id
GROUP BY p.id;

17.Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.

In [ ]:
WITH 
comp AS (SELECT id
         FROM company
         WHERE status = 'closed'
          AND id IN (SELECT company_id 
                     FROM funding_round
                     WHERE is_first_round = 1
                      AND is_last_round = 1)),
count_isnt AS (SELECT DISTINCT p.id,
                      COUNT(e.instituition) AS num_institutions
               FROM comp
               JOIN people AS p ON comp.id = p.company_id
               JOIN education AS e ON p.id = e.person_id
               GROUP BY p.id)
       
SELECT AVG(num_institutions) AS avg_num_institutions
FROM count_isnt;

18.Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники `Facebook*`.

`*(сервис, запрещённый на территории РФ)`

In [ ]:
WITH 
comp AS (SELECT id
         FROM company
         WHERE name = 'Facebook'
          ),
count_isnt AS (SELECT DISTINCT p.id,
                      COUNT(e.instituition) AS num_institutions
               FROM comp
               JOIN people AS p ON comp.id = p.company_id
               JOIN education AS e ON p.id = e.person_id
               GROUP BY p.id)
       
SELECT AVG(num_institutions) AS avg_num_institutions
FROM count_isnt;

19.Составьте таблицу из полей:
* `name_of_fund` — название фонда;
* `name_of_company` — название компании;
* `amount` — сумма инвестиций, которую привлекла компания в раунде.

В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

In [ ]:
SELECT f.name AS name_of_fund,
       c.name AS name_of_company,
       fr.raised_amount AS amount
FROM investment AS i
JOIN company AS c ON i.company_id = c.id
JOIN fund AS f ON i.fund_id = f.id
JOIN funding_round AS fr ON i.funding_round_id = fr.id
WHERE c.milestones > 6
  AND EXTRACT('year' FROM i.created_at) BETWEEN 2012 AND 2013;


20.Выгрузите таблицу, в которой будут такие поля:
* название компании-покупателя;
* сумма сделки;
* название компании, которую купили;
* сумма инвестиций, вложенных в купленную компанию;
* доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.

Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы. 

Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.

In [ ]:
SELECT co.name as buyer,
       ac.price_amount,
       c.name,
       c.funding_total,
       ROUND(ac.price_amount/c.funding_total) AS attitude
FROM acquisition AS ac
JOIN company AS c ON  ac.acquired_company_id = c.id 
JOIN company as co ON ac.acquiring_company_id = co.id 
WHERE ac.price_amount <> 0 AND c.funding_total <> 0
ORDER BY ac.price_amount DESC,c.name
LIMIT 10;


21.Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.

In [ ]:
SELECT c.name, 
       EXTRACT(MONTH FROM f.funded_at) as month
FROM company c
JOIN funding_round f ON c.id = f.company_id
WHERE c.category_code = 'social'
AND f.funded_at BETWEEN '2010-01-01' AND '2013-12-31'
AND f.raised_amount > 0
ORDER BY f.funded_at ASC

22.Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
* номер месяца, в котором проходили раунды;
* количество уникальных названий фондов из США, которые инвестировали в этом месяце;
* количество компаний, купленных за этот месяц;
* общая сумма сделок по покупкам в этом месяце.

In [ ]:
WITH funds AS 
(SELECT 
    COUNT(DISTINCT fu.name) AS funds_count,
    EXTRACT(MONTH FROM funded_at::DATE) AS MONTH
    --fr.raised_amount as round_cash
FROM investment AS inv
FULL OUTER JOIN funding_round AS fr ON inv.funding_round_id = fr.id
FULL OUTER JOIN fund AS fu ON inv.fund_id = fu.id
WHERE fr.funded_at::DATE BETWEEN '2010-01-01'::DATE AND '2013-12-31'::DATE
AND fu.country_code = 'USA'
GROUP BY MONTH
ORDER BY MONTH
),
 
aq AS (SELECT 
    EXTRACT(MONTH FROM acquired_at::DATE) AS MONTH,
    COUNT(acquired_company_id),
    SUM(price_amount)
  
FROM acquisition
WHERE acquired_at::DATE BETWEEN '2010-01-01'::DATE AND '2013-12-31'::DATE
GROUP BY MONTH)
 
SELECT
    aq.month,
    funds.funds_count,
    aq.count AS company_count,
    aq.sum AS prices_sum
FROM funds AS funds
JOIN aq ON aq.month = funds.month

23.Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

In [ ]:
WITH
t2011 AS (SELECT country_code,
                AVG(funding_total) AS avg_investments
          FROM company
          WHERE EXTRACT(YEAR FROM founded_at) = 2011
          GROUP BY country_code),
t2012 AS (SELECT country_code,
                 AVG(funding_total) AS avg_investments
          FROM company
          WHERE EXTRACT(YEAR FROM founded_at) = 2012
          GROUP BY country_code),
t2013 AS (SELECT country_code,
                AVG(funding_total) AS avg_investments
          FROM company
          WHERE EXTRACT(YEAR FROM founded_at) = 2013
          GROUP BY country_code)

SELECT t2011.country_code,
       t2011.avg_investments AS avg_2011,
       t2012.avg_investments AS avg_2012,
       t2013.avg_investments AS avg_2013
FROM t2011 
LEFT JOIN t2012 ON t2011.country_code = t2012.country_code
LEFT JOIN t2013 ON t2012.country_code = t2013.country_code
WHERE t2011.avg_investments IS NOT NULL
    AND t2012.avg_investments IS NOT NULL
    AND t2013.avg_investments IS NOT NULL
ORDER BY 2 DESC
